In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from io import BytesIO
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import joblib
import boto3
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# --- Step 1: Load Data from AWS S3 ---
bucket_name = 'my-feature-store-data'
data_key = 'pipeline-data/data.csv'

# Create S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
)

# Load dataset
obj = s3.get_object(Bucket=bucket_name, Key=data_key)
df = pd.read_csv(obj['Body'])

# Process date columns
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
df = df.dropna(subset=['aqi_index', 'Calculated_AQI'])  # Remove rows with missing target values

# Define features and target
target_columns = ['aqi_index', 'Calculated_AQI']
date_columns = ['year', 'month', 'day', 'hour']
features = [col for col in df.columns if col not in target_columns and col != 'date']

X = df[features]
y = df[target_columns]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# initialize Y_pred to None
Y_predicted = None
# --- Step 2: Load or Train Model ---
try:
    model_key = 'models/best_model.pkl'
    print(f"Attempting to load model from S3: {bucket_name}/{model_key}")
    
    # Check if model exists
    s3.get_object(Bucket=bucket_name, Key=model_key)
    print(f"Model file exists at {bucket_name}/{model_key}")
    # Download model
    model_buffer = BytesIO()
    with open(model_buffer, 'wb') as f:
        s3.download_fileobj(bucket_name, model_key, f)
    model_buffer.seek(0)
    # Load model
    print("Loading model...")
    model = joblib.load(model_buffer)
    print("Successfully loaded pre-trained model from S3") 
    y_pred = model.predict(X_test_scaled) 
    Y_predicted = y_pred
except Exception as e:
    print(f"Could not load model: {str(e)}. Training new model.")
    # Train new model
    model = RandomForestRegressor(n_estimators=300, random_state=42)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    Y_predicted = y_pred

# --- Step 3: Evaluate Model ---


print("\nModel Performance:")
for i, col in enumerate(target_columns):
    rmse = np.sqrt(mean_squared_error(y_test.iloc[:, i], Y_predicted[:, i]))
    mae = mean_absolute_error(y_test.iloc[:, i], Y_predicted[:, i])
    r2 = r2_score(y_test.iloc[:, i], Y_predicted[:, i])
    
    print(f"  Target: {col}")
    print(f"    RMSE: {rmse:.4f}")
    print(f"    MAE: {mae:.4f}")
    print(f"    R²: {r2:.4f}")

# --- Step 4: Predict AQI for Next 3 Days ---
start_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1)
future_dates = [start_date + timedelta(days=i) for i in range(3)]

# Prepare future features
future_features = pd.DataFrame({
    "year": [d.year for d in future_dates],
    "month": [d.month for d in future_dates],
    "day": [d.day for d in future_dates],
    "hour": [12] * 3,  # Predict for noon
})

# Find most recent data for April 2025
recent_data = df[(df['year'] == 2025) & (df['month'] == 4)]

numeric_features = [col for col in features if col not in date_columns]

if len(recent_data) > 0:
    recent_averages = recent_data[numeric_features].mean().to_dict()
else:
    similar_season_data = df[df['month'] == 4]
    if len(similar_season_data) > 0:
        recent_averages = similar_season_data[numeric_features].mean().to_dict()
    else:
        recent_averages = df[numeric_features].mean().to_dict()

for feature, value in recent_averages.items():
    future_features[feature] = value

# Add small variations to weather-related features
for i in range(1, len(future_features)):
    for feature in ['temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'wind_speed_10m', 'wind_direction_10m', 'surface_pressure']:
        if feature in future_features.columns:
            variation = np.random.uniform(-0.05, 0.05)
            future_features.loc[i, feature] = future_features.loc[i-1, feature] * (1 + variation)

# Fill missing columns
required_columns = X_train.columns.tolist()
for col in required_columns:
    if col not in future_features.columns:
        future_features[col] = X_train[col].mean()

# Order columns correctly
future_features = future_features[required_columns]

# Scale and predict
future_scaled = scaler.transform(future_features)
predictions = model.predict(future_scaled)

prediction_results = pd.DataFrame({
    "Date": [d.strftime("%Y-%m-%d") for d in future_dates],
    "Predicted_AQI": np.round(predictions[:, 0], 2),
    "Predicted_Calculated_AQI": np.round(predictions[:, 1], 2)
})

print("\nPredicted values for the next 3 days:")
print(prediction_results)

# --- Step 5: Feature Importance ---
if hasattr(model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': model.feature_importances_,
    }).sort_values('Importance', ascending=False)
    
    print("\nTop 10 most important features:")
    print(feature_importance.head(10))
elif hasattr(model, 'estimators_'):
    estimator = model.estimators_[0]
    if hasattr(estimator, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'Feature': X_train.columns,
            'Importance': estimator.feature_importances_,
        }).sort_values('Importance', ascending=False)
        
        print("\nTop 10 most important features (from first target model):")
        print(feature_importance.head(10))

# --- Step 6: Save Model and Scaler to S3 ---
# Save model
model_buffer = BytesIO()
joblib.dump(model, model_buffer)
model_buffer.seek(0)
s3.upload_fileobj(model_buffer, Bucket=bucket_name, Key='models/retrained_multioutput_model.pkl')
print("\nRetrained model saved to S3.")

# Save scaler
scaler_buffer = BytesIO()
joblib.dump(scaler, scaler_buffer)
scaler_buffer.seek(0)
s3.upload_fileobj(scaler_buffer, Bucket=bucket_name, Key='models/scaler.pkl')
print("Scaler saved to S3.")

# Save predictions locally
prediction_results.to_pickle("prediction_results.pkl")
print("Prediction results saved locally as 'prediction_results.pkl'.")


Attempting to load model from S3: my-feature-store-data/models/best_model.pkl


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator GradientBoostingRegressor from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DummyRegressor from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Could not load model: <class 'numpy.random._mt19937.MT19937'> is not a known BitGenerator module.. Training new model.

Model Performance:
  Target: aqi_index
    RMSE: 0.2403
    MAE: 0.1191
    R²: 0.9314
  Target: Calculated_AQI
    RMSE: 7.1535
    MAE: 1.5161
    R²: 0.9952

Predicted values for the next 3 days:
         Date  Predicted_AQI  Predicted_Calculated_AQI
0  2025-04-30           3.64                    232.20
1  2025-05-01           3.74                    233.26
2  2025-05-02           3.74                    234.68

Top 10 most important features:
           Feature  Importance
7             pm10    0.444113
1               co    0.379451
0            index    0.132354
6            pm2_5    0.018193
5              so2    0.009151
4               o3    0.006023
3              no2    0.004908
9   temperature_2m    0.001786
8              nh3    0.001293
12  wind_speed_10m    0.000457

Retrained model saved to S3.
Scaler saved to S3.
Prediction results saved locally as '

In [6]:
# Save `prediction_results` to a file
prediction_results.to_pickle("prediction_results.pkl")
